In [1]:
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential,Model,initializers,layers,Input
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
import os

In [2]:
import tensorflow as tf
import numpy as np

(x_train,y_train),(x_test,y_test)=tf.keras.datasets.cifar10.load_data()

# Normalization
x_train=x_train/255.

x_test=x_test/255.

x_train.shape

x_train = tf.image.resize(x_train, [299,299]) 

x_test = tf.image.resize(x_test, [299,299]) 

y_train = tf.keras.utils.to_categorical(np.array(y_train))
y_test = tf.keras.utils.to_categorical(np.array(y_test))

In [6]:
def Stem(x):
    x=layers.Conv2D(filters=32,kernel_size=(3,3),strides=(2,2))(x)
    x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)
    x=layers.Conv2D(filters=32,kernel_size=(3,3),strides=(1,1))(x)
    x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)
    x=layers.Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='same')(x)
    x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)
    x1=layers.MaxPooling2D(pool_size=(3,3),strides=(2,2))(x)
    x2=layers.Conv2D(filters=96,kernel_size=(3,3),strides=(2,2))(x)
    x2=layers.BatchNormalization()(x2)
    x2=layers.ReLU()(x2)
    x3=tf.keras.layers.Concatenate()([x1, x2])

    x4=layers.Conv2D(filters=64,kernel_size=(1,1),strides=(1,1))(x3)
    x4=layers.BatchNormalization()(x4)
    x4=layers.ReLU()(x4)
    x4=layers.Conv2D(filters=96,kernel_size=(3,3),strides=(1,1))(x4)
    x4=layers.BatchNormalization()(x4)
    x4=layers.ReLU()(x4)

    x5=layers.Conv2D(filters=64,kernel_size=(1,1),strides=(1,1))(x3)
    x5=layers.BatchNormalization()(x5)
    x5=layers.ReLU()(x5)
    x5=layers.Conv2D(filters=64,kernel_size=(7,1),strides=(1,1),padding='same')(x5)
    x5=layers.BatchNormalization()(x5)
    x5=layers.ReLU()(x5)
    x5=layers.Conv2D(filters=64,kernel_size=(1,7),strides=(1,1),padding='same')(x5)
    x5=layers.BatchNormalization()(x5)
    x5=layers.ReLU()(x5)
    x5=layers.Conv2D(filters=96,kernel_size=(3,3),strides=(1,1))(x5)
    x5=layers.BatchNormalization()(x5)
    x5=layers.ReLU()(x5)
    x6=tf.keras.layers.Concatenate()([x4, x5])

    x7=layers.MaxPooling2D(pool_size=(3,3),strides=(2,2))(x6)
    x8=layers.Conv2D(filters=192,kernel_size=(3,3),strides=(2,2))(x6)
    x8=layers.BatchNormalization()(x8)
    x8=layers.ReLU()(x8)
    x9=tf.keras.layers.Concatenate()([x7, x8])

    return x9


def Inception_ResNet_Block_A(x):
    x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)

    x1=layers.Conv2D(filters=32,kernel_size=(1,1),strides=(1,1),padding='same')(x)
    x1=layers.BatchNormalization()(x1)
    x1=layers.ReLU()(x1)

    x2=layers.Conv2D(filters=32,kernel_size=(1,1),strides=(1,1),padding='same')(x)
    x2=layers.BatchNormalization()(x2)
    x2=layers.ReLU()(x2)
    x2=layers.Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),padding='same')(x2)
    x2=layers.BatchNormalization()(x2)
    x2=layers.ReLU()(x2)

    x3=layers.Conv2D(filters=32,kernel_size=(1,1),strides=(1,1),activation='relu',padding='same')(x)
    x3=layers.Conv2D(filters=48,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(x3)
    x3=layers.Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(x3)

    x4=tf.keras.layers.Concatenate()([x,x1,x2,x3])
    x4=layers.Conv2D(filters=384,kernel_size=(1,1),strides=(1,1),padding='same')(x4)
    x4=layers.BatchNormalization()(x4)
    x4=layers.ReLU()(x4)
    x5=tf.keras.layers.Add()([x,x4])

    x5=layers.ReLU()(x5)
    return x5



def Inception_ResNet_Block_B(x):
    x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)    

    x1=layers.Conv2D(filters=192,kernel_size=(1,1),strides=(1,1),padding='same')(x)
    x1=layers.BatchNormalization()(x1)
    x1=layers.ReLU()(x1)

    x2=layers.Conv2D(filters=128,kernel_size=(1,1),strides=(1,1),padding='same')(x)
    x2=layers.BatchNormalization()(x2)
    x2=layers.ReLU()(x2)
    x2=layers.Conv2D(filters=160,kernel_size=(1,7),strides=(1,1),padding='same')(x2)
    x2=layers.BatchNormalization()(x2)
    x2=layers.ReLU()(x2)
    x2=layers.Conv2D(filters=192,kernel_size=(7,1),strides=(1,1),padding='same')(x2)
    x2=layers.BatchNormalization()(x2)
    x2=layers.ReLU()(x2)
    
    x3=tf.keras.layers.Concatenate()([x1,x2])

    x3=layers.Conv2D(filters=1152,kernel_size=(1,1),strides=(1,1),padding='same')(x3)
    x3=layers.BatchNormalization()(x3)
    x3=layers.ReLU()(x3)
    
    x4=tf.keras.layers.Add()([x,x3])

    x4=layers.ReLU()(x4)

    return x4


def Inception_ResNet_Block_C(x):
    x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)

    x1=layers.Conv2D(filters=192,kernel_size=(1,1),strides=(1,1),padding='same')(x)
    x1=layers.BatchNormalization()(x1)
    x1=layers.ReLU()(x1)

    x2=layers.Conv2D(filters=192,kernel_size=(1,1),strides=(1,1),padding='same')(x)
    x2=layers.BatchNormalization()(x2)
    x2=layers.ReLU()(x2)
    x2=layers.Conv2D(filters=224,kernel_size=(1,3),strides=(1,1),padding='same')(x2)
    x2=layers.BatchNormalization()(x2)
    x2=layers.ReLU()(x2)
    x2=layers.Conv2D(filters=256,kernel_size=(3,1),strides=(1,1),padding='same')(x2)
    x2=layers.BatchNormalization()(x2)
    x2=layers.ReLU()(x2)
    
    x3=tf.keras.layers.Concatenate()([x1,x2])

    x3=layers.Conv2D(filters=2048,kernel_size=(1,1),strides=(1,1),padding='same')(x3)
    x3=layers.BatchNormalization()(x3)
    x3=layers.ReLU()(x3)
    
    x4=tf.keras.layers.Add()([x,x3])

    x4=layers.ReLU()(x4)

    return x4


def Reduction_Block_A(x):
    x4=layers.MaxPooling2D(pool_size=(3,3),strides=(2,2))(x)

    x1=layers.Conv2D(filters=384,kernel_size=(3,3),strides=(2,2))(x)
    x1=layers.BatchNormalization()(x1)
    x1=layers.ReLU()(x1)

    x2=layers.Conv2D(filters=256,kernel_size=(1,1),strides=(1,1),padding='same')(x)
    x2=layers.BatchNormalization()(x2)
    x2=layers.ReLU()(x2)

    x2=layers.Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),padding='same')(x2)
    x2=layers.BatchNormalization()(x2)
    x2=layers.ReLU()(x2)
    x2=layers.Conv2D(filters=384,kernel_size=(3,3),strides=(2,2),activation='relu')(x2)
    x2=layers.BatchNormalization()(x2)
    x2=layers.ReLU()(x2)

    x3=tf.keras.layers.Concatenate()([x4,x1,x2])
    
    return x3 



def Reduction_Block_B(x):
    x1=layers.MaxPooling2D(pool_size=(3,3),strides=(2,2))(x)

    x2=layers.Conv2D(filters=256,kernel_size=(1,1),strides=(1,1),padding='same')(x)
    x2=layers.BatchNormalization()(x2)
    x2=layers.ReLU()(x2)
    x2=layers.Conv2D(filters=288,kernel_size=(3,3),strides=(2,2))(x2)
    x2=layers.BatchNormalization()(x2)
    x2=layers.ReLU()(x2)
    

    x3=layers.Conv2D(filters=256,kernel_size=(1,1),strides=(1,1),padding='same')(x)
    x3=layers.BatchNormalization()(x3)
    x3=layers.ReLU()(x3)
    x3=layers.Conv2D(filters=288,kernel_size=(3,3),strides=(2,2))(x3)
    x3=layers.BatchNormalization()(x3)
    x3=layers.ReLU()(x3)

    x4=layers.Conv2D(filters=256,kernel_size=(1,1),strides=(1,1),padding='same')(x)
    x4=layers.BatchNormalization()(x4)
    x4=layers.ReLU()(x4)
    x4=layers.Conv2D(filters=288,kernel_size=(3,3),strides=(1,1),padding='same')(x4)
    x4=layers.BatchNormalization()(x4)
    x4=layers.ReLU()(x4)
    x4=layers.Conv2D(filters=320,kernel_size=(3,3),strides=(2,2))(x4)
    x4=layers.BatchNormalization()(x4)
    x4=layers.ReLU()(x4)

    x5=tf.keras.layers.Concatenate()([x1,x2,x3,x4])
    
    return x5 

In [7]:

input=layers.Input(shape=(299,299,3))
x=Stem(input)
    
x=Inception_ResNet_Block_A(x)
x=Inception_ResNet_Block_A(x)
x=Inception_ResNet_Block_A(x)
x=Inception_ResNet_Block_A(x)
x=Inception_ResNet_Block_A(x)
    
x6=Reduction_Block_A(x)
    
x6=Inception_ResNet_Block_B(x6)#1
x6=Inception_ResNet_Block_B(x6)#2
x6=Inception_ResNet_Block_B(x6)#3
x6=Inception_ResNet_Block_B(x6)#4
x6=Inception_ResNet_Block_B(x6)#5
x6=Inception_ResNet_Block_B(x6)#6
x6=Inception_ResNet_Block_B(x6)#7
x6=Inception_ResNet_Block_B(x6)#8
x6=Inception_ResNet_Block_B(x6)#9
x6=Inception_ResNet_Block_B(x6)#10
    
x6=Reduction_Block_B(x6)
    
x6=Inception_ResNet_Block_C(x6)#1
x6=Inception_ResNet_Block_C(x6)#2
x6=Inception_ResNet_Block_C(x6)#3
x6=Inception_ResNet_Block_C(x6)#4
x6=Inception_ResNet_Block_C(x6)#5
    
x6=layers.GlobalAveragePooling2D()(x6)
x6=layers.Dropout(0.8)(x6)
x6=layers.Dense(units=1000,activation='softmax')(x6)
    
model = tf.keras.models.Model(input,x6)

In [4]:
model.build(input_shape=(299, 299, 3))

In [ ]:
plot_model(model,to_file='Inception_ResNet_v2.png',show_shapes=True)

In [27]:
len(model.layers)

479

In [8]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_132 (Conv2D)            (None, 149, 149, 32  896         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_137 (Batch  (None, 149, 149, 32  128        ['conv2d_132[0][0]']             
 Normalization)                 )                                                           

In [8]:
model.compile(optimizer='adam',loss='categorical_crossentropy' ,metrics=['acc'])

In [9]:
with tf.device("/device:GPU:0"):
    history=model.fit(train_ds,validation_data=validation_ds,epochs=10,batch_size=16,verbose=1)

 869/6250 [===>..........................] - ETA: 4:24:15 - loss: 2.3157 - acc: 0.0959